В решении задачи в качестве исходного дата-сета используйте CSV-файл https://video.ittensive.com/analytics-way/2022-04-22.dataset.csv.

На входе у вас есть таблица orders, содержащая все заказы, совершенные в ОЗОН<br>
Описание таблицы orders:<br>
* user_id (string) - уникальный идентификатор пользователя
* order_id (string) - уникальный идентификатор заказа
* order_time (Int64) - время сделанного заказа в unixtime (секунды, UTC)
* order_cost (float) - стоимость заказа
* success_order_flg (bool) - идентификатор, определяющий был ли заказ успешно доставлен. Выполненный заказ - это заказ у которого в данной таблице success_order_flg=True

Пользователь считается привлеченным в день date, если он совершает первый заказ в день date.

Пользователь считается реактивированным в день date, если он совершает заказ в день date, при этом не совершил ни одного заказа в течении предшествующих 90 дней, но совершал хотя бы 1 заказ за свою пользовательскую историю ранее.

На выходе требуется получить таблицу:
* date - календарная дата привлечения или реактивации
* gmv360d_new - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, привлеченными в день date
* gmv360d_reactivated - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, реактивированными в день date
* users_count_new - кол-во пользователей, привлеченных в день date
* users_count_reactivated- кол-во пользователей, реактивированных в день date

Заказы, совершенные пользователем после очередной реактивации, не должны учитываться в расчете GMV для предыдущих периодов активности пользователя. Т.е., например, если пользователь пришел в день 0, совершал заказы по день 30, не совершал заказов до дня 130 и совершил заказ в день 130, его заказы в дни 130-490 должны учитываться только в gmv360d_reactivated для дня 130, но не в gmv360d_new дня 0.

То же правило распространяется на повторные реактивации пользователя. То есть если у рассмотренного нами пользователя на 230-й день жизни снова произошла реактивация, то gmv360d_reactivated будет посчитано для предыдущего периода за дни 130-229, а для итогового, с реактивацией на 230-й день, это поле будет рассчитано за дни 230-590.

Сколько пользователей было реактивировано 20 марта 2020 года?

In [1]:
# загрузка необходимых библиотек

import pandas as pd
from datetime import datetime

Загрузка данных, добавление даты заказа в order_data

In [3]:
data = pd.read_csv('https://video.ittensive.com/analytics-way/2022-04-22.dataset.csv', sep=';')
print(data.head(3))

   user_id   order_id  order_time  order_cost  success_order_flg
0  1728898  118881751  1578345249  12804.1667                  0
1  1728898  132150404  1584367397   5147.2728                  1
2  1728898  133654929  1585501200   8600.0000                  1


Добавим новую серию данных дата заказа.

In [4]:
data['order_date'] = pd.to_datetime(data['order_time'] ,unit='s').dt.date.astype('datetime64[D]')
data.head(3)

,user_id,order_id,order_time,order_cost,success_order_flg,order_date
0,1728898,118881751,1578345249,12804.1667,0,2020-01-06
1,1728898,132150404,1584367397,5147.2728,1,2020-03-16
2,1728898,133654929,1585501200,8600.0000,1,2020-03-29


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56680 entries, 0 to 56679
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            56680 non-null  int64         
 1   order_id           56680 non-null  int64         
 2   order_time         56680 non-null  int64         
 3   order_cost         56680 non-null  float64       
 4   success_order_flg  56680 non-null  int64         
 5   order_date         56680 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 2.6 MB


Пользователь считается реактивированным в день date, если он совершает заказ в день date, при этом не совершил ни одного заказа в течении предшествующих 90 дней, но совершал хотя бы 1 заказ за свою пользовательскую историю ранее.

На выходе требуется получить таблицу:
* date - календарная дата привлечения или реактивации
* gmv360d_new - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, привлеченными в день date
* gmv360d_reactivated - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, реактивированными в день date
* users_count_new - кол-во пользователей, привлеченных в день date
* users_count_reactivated- кол-во пользователей, реактивированных в день date

Сколько пользователей было реактивировано 20 марта 2020 года?

In [7]:
day = '2020-03-20'
day2 = '2019-12-21'
# пользователи которые совершили хотя бы 1 заказ до дня реактивации day
# user_order_yes = data.loc[((data['order_date'] <= day) & (data['success_order_flg'] == 1))] 
user_order_yes = list(data.loc[((data['order_date'] <= day) & (data['success_order_flg'] == 1))]['user_id'].unique())
print(f'{len(user_order_yes)} - количество пользователей которые совершили хотя бы 1 заказ, до даты реактивации {day}.')

9471 - количество пользователей которые совершили хотя бы 1 заказ, до даты реактивации 2020-03-20.


In [19]:
# не совершил ни одного заказа в течении предшествующих 90 дней дню day
user_order_90day = list(data.loc[(data['order_date'] >= day2) & (data['order_date'] <= day) & (data['success_order_flg'] == 1)]['user_id'].unique())
user_no_order_90day = []
for i in range(len(user_order_yes)):
    if user_order_yes[i] not in user_order_90day:  # если не совершал в 90 дней, но совершал ранее
        user_no_order_90day.append(user_order_yes[i])  # добавляем в список не совершавших в 90 дней
print(f'{len(user_no_order_90day)} - количество пользователей, которое не совершали заказы за прошедшие 90 дней до даты реактивации {day}, хотя делали это ранее.')

0 - количество пользователей, которое не совершали заказы за прошедшие 90 дней до даты реактивации 2020-03-20, хотя делали это ранее.


In [242]:
# пользователи совершившие заказ в день day
user_order_yes_in_day = list(data.loc[((data['order_date'] == day) & (data['success_order_flg'] == 1))]['user_id'].unique())
print(f'{len(user_order_yes_in_day)} - пользователя совершили заказ на дату реактивации {day}')

282 - пользователя совершили заказ на дату реактивации 2020-03-20


In [243]:
# users_count_new - кол-во пользователей, привлеченных в день date
users_count_new = 0
for i in range(len(user_order_yes_in_day)):
    if user_order_yes_in_day[i] not in user_order_yes:
        users_count_new += 1
print(f'{users_count_new} - пользователей было привлечено в дату {day}.')

0 - пользователей было привлечено в дату 2020-03-20.


#### Сколько пользователей было реактивировано 20 марта 2020 года?

In [244]:
users_count_reactivated = 0
for i in range(len(user_no_order_90day)):
    if user_no_order_90day[i] in user_order_yes:
        users_count_reactivated += 1
print(f'{users_count_reactivated} - пользователей было реактивировано в дату реактивации {day}.')

0 - пользователей было реактивировано в дату реактивации 2020-03-20.
